<a href="https://colab.research.google.com/github/giuliovv/ANNDL_competition_1/blob/master/giulio_models/inception_noise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os

colab = "True" #@param ['True','False']
if colab == "True":
  from google.colab import drive
  drive.mount('/gdrive')
  %cd /gdrive/MyDrive/Colab Notebooks
  if not os.path.isdir('training'):
    !unzip dataset.zip

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive/MyDrive/Colab Notebooks


In [2]:
import os
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

In [3]:
dataset_dir = 'training'

Let the game begin

In [4]:
import tensorflow as tf

In [5]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from keras.preprocessing.image import ImageDataGenerator

In [6]:
from sklearn.model_selection import train_test_split

In [7]:
labels = ['Apple','Blueberry','Cherry','Corn','Grape','Orange','Peach','Pepper','Potato','Raspberry','Soybean','Squash','Strawberry','Tomato']

In [8]:
y = tf.keras.utils.to_categorical(range(len(labels)))
y

array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]],
      dtype=float32)

In [9]:
import shutil

get_plants = False

if not "test" in os.listdir():
  print("No test directory!")
  for label in labels:
    print(label)
    image_names = [pic for pic in os.listdir("training/"+label)]
    train_images, test_images = train_test_split(image_names, test_size=0.05)
    if not 'test/'+label in os.listdir():
      try:
        os.makedirs(os.path.abspath(os.getcwd())+'/test/'+label+'/')
      except OSError:
        print("Failed to create a dir for ", '/test/'+label+'/')
      else:
        print("Success creating folder ", label)
    for name in test_images:
      shutil.move(os.path.abspath(os.getcwd())+'/training/'+label+'/'+name, os.path.abspath(os.getcwd())+'/test/'+label+'/'+name)
  print("Transfered all testing data!")

if get_plants:
  ! pip install -q tfds-nightly
  import tensorflow_datasets as tfds
  from PIL import Image
  ds = tfds.load('plant_village', split='train')
  builder = tfds.builder('plant_village')
  info = builder.info
  labels_tf = info.features["label"].names
  if not 'test_expanded/' in os.listdir():
      try:
        os.makedirs(os.path.abspath(os.getcwd())+'/test_expanded/')
      except OSError:
        print("Failed to create a dir for /test_expanded/")
      else:
        print("Success creating folder test_expanded")
  image_number = 54303
  ds = ds.take(54303)
  for el in ds:
    label_new_ds = labels_tf[el["label"].numpy()]
    for just_fruit_name in labels:
      if just_fruit_name in label_new_ds:
        if not just_fruit_name in os.listdir("test_expanded"):
          try:
            os.makedirs(os.path.abspath(os.getcwd())+'/test_expanded/'+just_fruit_name+'/')
          except OSError:
            print("Failed to create a dir for ", '/test_expanded/'+just_fruit_name+'/')
          else:
            print("Success creating folder ", just_fruit_name)
        im = Image.fromarray(el["image"].numpy())
        im.save(os.path.abspath(os.getcwd())+'/test_expanded/'+just_fruit_name+"/"+str(image_number)+".jpeg")
        break
    if image_number % 1000 == 0:
      print(image_number)
    image_number += 1


In [10]:
import random

def preproc(img):
  '''Add random noise to an image'''
  VARIABILITY = 50
  deviation = VARIABILITY*random.random()
  noise = np.random.normal(0, deviation, img.shape)
  img += noise
  np.clip(img, 0., 255.)
  X = tf.keras.applications.inception_v3.preprocess_input(img)
  return X

In [11]:
BATCH_SIZE = 64
VALIDATE_BATCH_SIZE = BATCH_SIZE

train_generator = ImageDataGenerator(
## when rotating some pixels remain unfilled, to fill them the default function is to take the "nearest pixel color", this maybe screws up the learning process so I left it for later training
                                    rotation_range=10,
                                    fill_mode='nearest',
                                    brightness_range=(0.6,1.4),
                                    channel_shift_range=150,
                                    shear_range=0.2,
                                    zoom_range=0.2,
                                    width_shift_range=0.1, 
                                    height_shift_range=0.1,
                                    horizontal_flip=True, 
                                    vertical_flip=True,
                                    validation_split=0.2,
                                    preprocessing_function=preproc) # VGG16 preprocessing
test_generator = ImageDataGenerator(preprocessing_function=tf.keras.applications.inception_v3.preprocess_input)

traingen = train_generator.flow_from_directory('training',
                                              target_size=(256, 256),
                                              class_mode='categorical',
                                              classes=labels,
                                              subset='training',
                                              batch_size=BATCH_SIZE,
                                              shuffle=True,
                                              seed=42)

validgen = train_generator.flow_from_directory('training',
                                              target_size=(256, 256),
                                              class_mode='categorical',
                                              classes=labels,
                                              subset='validation',
                                              batch_size=VALIDATE_BATCH_SIZE,
                                              shuffle=True,
                                              seed=42)

testgen = test_generator.flow_from_directory('test',
                                            target_size=(256, 256),
                                            class_mode='categorical',
                                            classes=labels,
                                            batch_size=1,
                                            shuffle=False,
                                            seed=42)

Found 13238 images belonging to 14 classes.
Found 3301 images belonging to 14 classes.
Found 877 images belonging to 14 classes.


In [12]:
# create the base pre-trained model
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(256, 256,3))

In [13]:
n_classes = len(labels)

In [ ]:
load = True

if load:
  model = tf.keras.models.load_model("inception3_only_top_noise")

In [ ]:
if not load:
  # add a global spatial average pooling layer
  x = base_model.output
  x = GlobalAveragePooling2D()(x)
  # let's add a fully-connected layer
  x = Dense(1024, activation='relu')(x)
  # and a logistic layer -- let's say we have n_classes classes
  predictions = Dense(n_classes, activation='softmax')(x)

  # this is the model we will train
  model = Model(inputs=base_model.input, outputs=predictions)


In [ ]:
if not load:

  # first: train only the top layers (which were randomly initialized)
  # i.e. freeze all convolutional InceptionV3 layers
  for layer in base_model.layers:
      layer.trainable = False

  optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5)

  # compile the model (should be done *after* setting layers to non-trainable)
  model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
callbacks = []

early_stop = True
if early_stop:
    es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=6)
    callbacks.append(es_callback)
    
#LRPlateau
lr_plateau_callback = tf.keras.callbacks.ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.2,
    patience=3,
    min_lr=0,
)  

callbacks.append(lr_plateau_callback)

tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="./logs", histogram_freq=1)

callbacks.append(tensorboard_callback)

backup = tf.keras.callbacks.experimental.BackupAndRestore(
    "top_only_noise"
)

In [ ]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir "./logs"

In [ ]:
if not load:
  # train the model on the new data for a few epochs
  model.fit(traingen, epochs=10, steps_per_epoch=10, batch_size=128, validation_data=validgen, callbacks=callbacks+[backup])

In [ ]:
if not load:
  model.save("inception3_only_top_noise")

In [ ]:
# at this point, the top layers are well trained and we can start fine-tuning
# convolutional layers from inception V3. We will freeze the bottom N layers
# and train the remaining top layers.

# let's visualize layer names and layer indices to see how many layers
# we should freeze:
for i, layer in enumerate(base_model.layers):
   print(i, layer.name)

In [ ]:
load_second = False
if load_second:
  model = tf.keras.models.load_model("inception3")

In [ ]:
if not load_second:
  # we chose to train the top 2 inception blocks, i.e. we will freeze
  # the first 249 layers and unfreeze the rest:
  for layer in model.layers[:50]:
    layer.trainable = False
  for layer in model.layers[50:]:
    layer.trainable = True

  # we need to recompile the model for these modifications to take effect
  # we use SGD with a low learning rate
  from tensorflow.keras.optimizers import Adam
  model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

backup2 = tf.keras.callbacks.experimental.BackupAndRestore(
    "second_part_noise"
)

# we train our model again (this time fine-tuning the top 2 inception blocks
# alongside the top Dense layers
model.fit(traingen, epochs=25, steps_per_epoch=10,  batch_size=128, validation_data=validgen, callbacks=callbacks+[backup2])

In [ ]:
model.save("inception3_noise")

In [ ]:
model.evaluate(testgen)